In [ ]:
import numpy as np
import pandas as pd
import gdstk
import matplotlib.pyplot as plt
from cytoolz import unique

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.microfluidics_design as microfluidics_design
import paulssonlab.microfluidics_design.fluid as fluid
import paulssonlab.microfluidics_design.fov as fov

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

mu = 0.7 * u.cP  # dynamic viscosity
nu = mu / (1 * u.g / u.cm**3)  # kinematic viscosity

In [ ]:
camera_dims = {
    "iris15": (4.25, np.array([5056, 2960])),
    "iris9": (4.25, np.array([2960, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
mags = (20, 40)
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in mags:
        fov_dims[f"{camera} {mag}x"] = np.floor(fov_dim * pixel_size / mag).astype(np.int32)

In [ ]:
def print_flow_info(height, flow_rate, mu, nu, metadata):
    for chip_name, md in metadata.items():
        flow_info = fluid.snake_flow(
            height,
            md,
            length_unit=u.um,
        )
        print(f"{chip_name}:")
        for input_name, input_flow in flow_info.items():
            R = input_flow["R"] * mu
            delta_P = R * flow_rate
            print(f"    {input_name}: {delta_P.to(u.bar):.2f}")
            print(
                f"        flow rate: {flow_rate:.0f~#P} ({flow_rate.to(u.milliliters/u.hour):.1f~#P}, {flow_rate.to(u.milliliters/u.day):.0f~#P})"
            )
            if "flow_nonuniformity" in input_flow:
                print(
                    f"        flow nonuniformity: {input_flow['flow_nonuniformity']:.2f}"
                )
            if "manifold_width" in md:
                L_e = fluid.entrance_length(
                    height,
                    md["manifold_width"] * u.um,
                    flow_rate,
                    nu,
                )
                print(f"        entrance length: {L_e}")

# Designs

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
# chip_dims = np.array([33e3, 18e3])
chip_dims0 = np.array([33e3, 18e3])
# chip_dims0 = np.array([23e3, 13e3])
# chip_dims = np.array([38e3, 19e3])
chip_dims = np.array([40e3, 19e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(
    plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False, color="red", lw=2)
)
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
# ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims / 2, *chip_dims, fill=False, lw=2))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
%%time
metadata = {}
base_params = dict(
    design_func=microfluidics_design.snake,
    dims=chip_dims,
    trench_length=35,
    feeding_channel_width=35,
    trench_gap=15,
    # trench_width=1.4,
    trench_spacing=2.1,
    trench_margin=0,
    border_margin=0.5e3,
    port_margin=1e3,
    port_radius=750 / 2,
    # port_radius=0,
    port=False,
    registration_marks="qr",
    registration_mark_barcodes=False,
    trenches=True,
    metadata=metadata,
)
manifold_params = {
    **base_params,
    **dict(
        design_func=microfluidics_design.manifold_snake,
        lanes_per_snake=3,
        manifold_input_style="bend-out",
        manifold_split=4,
        manifold_width=50,
        manifold_input_margin=775,  # 2e3,
        manifold_bend_margin=True,  # 0.2e3,
        manifold_bend_radius=0.1e3,
        manifold_margin=100,
    ),
}

wafer_id = 1
split_manifold_chip = dict(trench_width=1.4)
split_poor_manifold_chip = dict(
    manifold_trench_params=[
        dict(trench_width=1.3),
        dict(trench_width=1.3),
        dict(trench_width=1.5),
        dict(trench_width=1.5),
    ]
)

params = [split_manifold_chip, split_manifold_chip, split_poor_manifold_chip]
params = [
    {
        **manifold_params,
        **p,
        "ur_corner_label": f"{wafer_id}.{chip_id}",
    }
    for chip_id, p in enumerate(params, 1)
]
# chip names
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width_} TS{trench_spacing}{reg} TG{trench_gap}".format(
        ports=f"S{p['split']}"
        if "split" in p and np.isscalar(p["split"])
        else "S{} LS{}".format(p["manifold_split"], p["lanes_per_snake"]),
        reg=f" M{p.get('mark_size')} ID{p.get('chip_id')}"
        if p.get("registration_mark_barcodes")
        else "",
        trench_width_=p["trench_width"]
        if "trench_width" in p
        else "/".join(
            unique([str(d["trench_width"]) for d in p["manifold_trench_params"]])
        ),
        **p,
    )
    for p in params
]
# make chip cells
chips = [microfluidics_design.chip(name, **p) for p, name in zip(params, chip_names)]
# calculate FOV info and overlay FOVs on chip cells
fov_grid_df = {}
draw_fov_grid = True
for chip_name, chip in zip(chip_names, chips):
    chip_fov_grid_df = fov.get_fov_grids_df(
        fov_dims, metadata[chip_name], center_margins=False, skip=None
    )
    fov_grid_df[chip_name] = chip_fov_grid_df
    if draw_fov_grid:
        fov_grid_to_draw = chip_fov_grid_df.xs(pd.IndexSlice["iris15 20x",:,:],drop_level=False)
        #fov_grid_to_draw = chip_fov_grid_df
        fov.draw_fov_grid(
            chip,
            metadata[chip_name],
            fov_grid_to_draw,
            rotate=False,
        )
fov_grid_df = pd.concat(
    fov_grid_df,
    names=["chip"],
)
# lay out wafer, output design
wafer_manifest = "\n".join([f"{idx+1}) " + name for idx, name in enumerate(chip_names)])
wafer_diameter = 3 * u.inch
alignment_mark_position = wafer_diameter / 2 - 0.2 * u.inch
main_cell = microfluidics_design.wafer(
    chips,
    label_right="Basilisk\n JQS/DE/YG 230402",
    label_left=wafer_manifest,
    label=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=wafer_diameter.to(u.um).magnitude,
    alignment_mark_position=alignment_mark_position.to(u.um).magnitude,
)
MAX_POINTS = 2000  # LayoutEditor uses 8191, gdstk default is 199
microfluidics_design.write_gds(
    main_cell, "designs/230402basilisk_s4.gds", max_points=MAX_POINTS
)

In [ ]:
fov_grid_to_draw

In [ ]:
chip_fov_grid_df

In [ ]:
print(wafer_manifest)

In [ ]:
{k: [x["num_trenches"] for x in v["trench_info"].values()] for k, v in metadata.items()}

In [ ]:
{k: v["num_trenches"] for k, v in metadata.items()}

In [ ]:
{k: [x["num_lanes"] for x in v["input_info"].values()] for k, v in metadata.items()}

In [ ]:
{k: v["num_lanes"] for k, v in metadata.items()}

In [ ]:
# Q = 300 * u.uL / u.min
Q = 75 * u.uL / u.min
height = 70 * u.um
print_flow_info(height, Q, mu, nu, metadata)

In [ ]:
grid_df.loc[pd.IndexSlice[:, ["iris15 20x", "iris9 20x", "bsi 40x"]], :].round(2)

# Fluids

# FOVs

In [ ]:
# skip_first -> offset_active_regions = 0, -1, +1 (in draw_fov_grid)
# give offset sequence for each graph cycle and each offset_active_regions

In [ ]:
metadata.keys()

In [ ]:
chip_name = "Basilisk S4 LS3 FC30 L35 W1.4 TS2.1 TG15"
fov_name = "iris15 20x"

In [ ]:
(np.array([0.5, 1.0])).astype(np.int32)

In [ ]:
trench_md = metadata[chip_name]["trench_info"][0]
trench_md

In [ ]:
# region_heights = [35, 30, 35, 15]
# region_heights = [35, 40, 35, 20]
# region_heights = [35, 40, 35, 20, 20, 20]
region_heights = [35, 90, 35, 20]
total_height = sum(region_heights)

In [ ]:
fov_dims[fov_name][1]

In [ ]:
packings = fov.get_fov_packings(fov_dims[fov_name][1], region_heights)

In [ ]:
graph = fov._get_fov_graph(packings, total_height)

In [ ]:
import networkx as nx

In [ ]:
nx.draw(graph, with_labels=True)

In [ ]:
list(nx.simple_cycles(graph))

In [ ]:
packings

In [ ]:
fov.fov_plot([35, 71, 160], fov_dims[fov_name][1], region_heights)

In [ ]:
fov_dims[fov_name][1]

In [ ]:
755-180

In [ ]:
fov.fov_plot([160, 755+35, (755+35)+(700-71), (755+35)+(700-71)], fov_dims[fov_name][1], region_heights)

In [ ]:
fov.fov_plot([160,71], fov_dims[fov_name][1], region_heights)

In [ ]:
fov.fov_plot([35.5, 575.5, 1115.5, 1115.5+540], fov_dims[fov_name][1], region_heights)

In [ ]:
fov_grids = fov.get_fov_grids(
    fov_dims[fov_name][1],
    region_heights,
    center_margins=True,
    skip=None,
)
fov_grids

In [ ]:
fov.fov_plot_offsets(fov_grids[5], fov_dims[fov_name][1], region_heights, num=8)

In [ ]:
fov.get_fov_plot_ys(fov_grids[5])

In [ ]:
fov.fov_plot([35.5, 575.5, 1206.0, 1835.5], fov_dims[fov_name][1], region_heights)

In [ ]:
fov.fov_plot_offsets(fov_grids[2], fov_dims[fov_name][1], region_heights, num=6)

In [ ]:
fov.fov_plot([35.5, 575.5, 1115.5, 1115.5+540], fov_dims[fov_name][1], region_heights)

In [ ]:
metadata.keys()

In [ ]:
chip_name = 'Basilisk S4 LS3 FC35 L35 W1.3/1.5 TS2.1 TG15'

In [ ]:
d = fov.get_fov_grids_df(fov_dims, metadata[chip_name])

In [ ]:
d.xs(pd.IndexSlice["iris15 20x",:,:],drop_level=False)